In [1]:
%matplotlib inline

In [2]:
import sys, os
SciAnalysis_PATH='/home/group/Software/SciAnalysis/'
SciAnalysis_PATH in sys.path or sys.path.append(SciAnalysis_PATH)

In [3]:
import glob
from SciAnalysis import tools
from SciAnalysis.XSAnalysis.Data import *
from SciAnalysis.XSAnalysis import Protocols

In [4]:
%run -i  /home/group/Software/SciAnalysis/SciAnalysis/XSAnalysis/Protocols.py

In [5]:
# Experimental parameters
########################################
calibration = Calibration(wavelength_A=0.770088) # 16.1 keV
#calibration = Calibration(wavelength_A=0.619920987) # 20.0 keV
calibration.set_image_size(981, height=1043) # Pilatus1M
calibration.set_pixel_size(pixel_size_um=172.0)
calibration.set_distance(8.320) # 8.300 nominal detector distance
calibration.set_beam_position(490.0, 656.0)
mask_dir = SciAnalysis_PATH + '/SciAnalysis/XSAnalysis/masks/'
mask = Mask(mask_dir+'Dectris/Pilatus1M_main_gaps-mask.png')
#mask.load(mask_dir + 'Dectris/mask.png')

In [6]:
bs = '/home/yuzhang/Test_SciAnalysis/XS/'
source_dir = bs + 'data/'
output_dir = bs + 'results/'

In [7]:
load_args = { 'calibration' : calibration, 
             'mask' : mask,
             }
run_args = { 'verbosity' : 3,
            }
process = Protocols.ProcessorXS(load_args=load_args, run_args=run_args)

In [8]:
fp  = 'AJ_exp1_x-0.4225_y0.1056_0.50s_299317.tif'

infile = source_dir + fp

print( infile )

/home/yuzhang/Test_SciAnalysis/XS/data/AJ_exp1_x-0.4225_y0.1056_0.50s_299317.tif


In [10]:
protocols = [
    #linecut_qr_fit(show_region=False, show=False, qz=0.0285, dq=0.006, q0=0.0030, sigma=0.0010, fit_range=[0.00, 0.010], plot_range=[0, 0.050, 0, None], trim_range=[0, 0.05]) ,
    

    #linecut_angle_fit_custom(q0=0.0135, dq=0.0035, show=False, show_region=False, toh5=True, overwrite_h5 = True ) ,   
    #get_linecut_angle(q0=0.0135, dq=0.0035, show=False, show_region=False) ,       
    
    
    circular_average(ylog=True, plot_range=[0, 0.15, None, None], toh5=True ) , 
    #circular_average_sum(ylog=True, plot_range=[0, 0.15, None, None]  ) ,     
    Protocols.thumbnails(crop=None, resize=1.0, blur=None, cmap=cmap_vge, toh5=True) , 
    circular_average_q2I(show=False, q0=0.0140, qn_power=2.5, sigma=0.0008, 
                             plot_range=[0, 0.06, 0, None], fit_range=[0.008, 0.022], toh5=True) ,    
    circular_average_q2I_fit(show=False, q0=0.0140, qn_power=2.5, sigma=0.0008, 
                             plot_range=[0, 0.06, 0, None], fit_range=[0.008, 0.022], toh5=True) ,
        
    
    
    #Protocols.metadata_extract(patterns=patterns) ,
    ]

process.run([infile], protocols, output_dir=output_dir, force=True)

Running circular_average for AJ_exp1_x-0.4225_y0.1056_0.50s_299317
Running thumbnails for AJ_exp1_x-0.4225_y0.1056_0.50s_299317
Running circular_average_q2I for AJ_exp1_x-0.4225_y0.1056_0.50s_299317
Running circular_average_q2I_fit for AJ_exp1_x-0.4225_y0.1056_0.50s_299317


## Load the saved h5 file

In [11]:
hfile =   output_dir + 'AJ_exp1_x-0.4225_y0.1056_0.50s_299317.h5'

In [12]:
h5r = h5todict( hfile )

In [13]:
h5r.keys()

dict_keys(['circular_average', 'circular_average_q2I', 'circular_average_q2I_fit', 'thumbnails'])

In [14]:
h5r['circular_average']

{'data': array([[7.63043023e-04, 3.01250000e+01, 8.43363182e-05, 1.33065251e+01],
        [9.36020321e-04, 5.75555556e+01, 5.16622996e-05, 2.51676917e+01],
        [1.08630606e-03, 1.13818182e+02, 4.38901682e-05, 5.44714747e+01],
        ...,
        [1.37652152e-01, 1.42857143e-01, 4.88277024e-05, 2.02041896e-01],
        [1.37825280e-01, 3.00000000e-01, 4.76109034e-05, 3.27817717e-01],
        [1.38015949e-01, 1.66666667e-01, 5.73289079e-05, 3.91101572e-01]]),
 'label': array(['q', 'I(q)', 'q_err', 'I(q)_err'], dtype='<U8'),
 'results': {}}

# An example to save a dict as h5

In [15]:
d = { 'data': np.arange(10), 'label':'data', 'results': {  'collected time': '2019/10/10', 'fit_res': { 'p0': 10 } } }

In [16]:
dicttoh5( d, 'test.h5', h5path='/cms/', mode='a')

key (/cms/data) already exists. Not overwriting.
key (/cms/label) already exists. Not overwriting.
key (/cms/results/collected time) already exists. Not overwriting.
key (/cms/results/fit_res/p0) already exists. Not overwriting.


In [17]:
h5todict( 'test.h5' )

{'cms': {'data': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
  'label': 'data',
  'results': {'collected time': '2019/10/10', 'fit_res': {'p0': array(10)}}},
 'data': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 'label': 'data',
 'results': {'collected time': '2019/10/10', 'fit_res': {'p0': array(10)}}}